<a href="https://colab.research.google.com/github/iamankan/fruits-ml/blob/conference1/FruitmlTL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
w, h = (224, 224)
base_model = keras.applications.VGG16(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(w, h, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
#Freeze the base model
base_model.trainable = False

In [ ]:
inputs = keras.Input(shape=(w, h, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base_model(inputs)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(1024, activation='relu')(x)
outputs = keras.layers.Dense(6, activation='softmax')(x)
model = keras.Model(inputs, outputs)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
BATCH_SIZE = 32

train_generator = ImageDataGenerator()

val_generator = ImageDataGenerator()

test_generator = ImageDataGenerator()

In [ ]:
train_data_dir = '/content/drive/MyDrive/fruit-ml/fruit_data/train'
test_data_dir = '/content/drive/MyDrive/fruit-ml/fruit_data/test'
val_data_dir = '/content/drive/MyDrive/fruit-ml/fruit_data/validation'

In [ ]:
traingen = train_generator.flow_from_directory(train_data_dir,
                                               target_size=(w, h),
                                               class_mode='categorical',
#                                                classes=class_subset,
#                                                subset='training',
                                               batch_size=BATCH_SIZE, 
                                               shuffle=True,
                                               seed=42)

validgen = val_generator.flow_from_directory(val_data_dir,
                                               target_size=(w, h),
                                               class_mode='categorical',
#                                                classes=class_subset,
#                                                subset='validation',
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               seed=42)

testgen = test_generator.flow_from_directory(test_data_dir,
                                             target_size=(w, h),
                                             class_mode='categorical',
#                                              classes=class_subset,
                                             batch_size=1,
                                             shuffle=False,
                                             seed=42)

Found 11629 images belonging to 6 classes.
Found 1289 images belonging to 6 classes.
Found 5534 images belonging to 6 classes.


In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='/content/drive/MyDrive/fruit-ml/checkpoint_file.h5',
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)


model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.CategoricalCrossentropy(),
              metrics=[keras.metrics.CategoricalAccuracy()])
history = model.fit(traingen, epochs=20, validation_data= validgen,
              callbacks=[model_checkpoint_callback])



Epoch 1/20
364/364 [==============================] - 169s 462ms/step - loss: 0.0866 - categorical_accuracy: 0.9987 - val_loss: 2.8151 - val_categorical_accuracy: 0.9837
Epoch 2/20
364/364 [==============================] - 162s 444ms/step - loss: 0.4887 - categorical_accuracy: 0.9960 - val_loss: 4.4487 - val_categorical_accuracy: 0.9783
Epoch 3/20
364/364 [==============================] - 161s 443ms/step - loss: 0.1369 - categorical_accuracy: 0.9982 - val_loss: 2.9418 - val_categorical_accuracy: 0.9907
Epoch 4/20
364/364 [==============================] - 162s 444ms/step - loss: 0.1717 - categorical_accuracy: 0.9985 - val_loss: 3.7887 - val_categorical_accuracy: 0.9845
Epoch 5/20
364/364 [==============================] - 162s 446ms/step - loss: 0.0643 - categorical_accuracy: 0.9995 - val_loss: 3.2562 - val_categorical_accuracy: 0.9891
Epoch 6/20
364/364 [==============================] - 163s 448ms/step - loss: 4.6087e-04 - categorical_accuracy: 0.9999 - val_loss: 2.5567 - val_categ

In [ ]:
import matplotlib.pyplot as plt
import numpy

# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()